### Libraries

In [33]:
import os
import numpy as np
import tensorflow as tf
#from tensorflow.keras import mixed_precision
#from tensorflow.keras.optimizers import AdamW
from tensorflow_addons.optimizers import AdamW
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib.animation import FuncAnimation 
from mpl_toolkits.mplot3d import Axes3D
import time
import copy
import datetime


### Config

In [34]:
time_stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
from_scratch=False # True: start fresh, False: resume from checkpoint
# Define model saving paths
SAVE_Full_MODEL_PATH = f"/Users/akhidre/pubgit/CS230_T2motion/models/full_checkpoint.keras" # use this for full model checkpoint

# TensorBoard directory

log_dir = f"/Users/akhidre/pubgit/CS230_T2motion/fit/{time_stamp}/"

#define data destination
TRAIN_NPZ = "/Users/akhidre/pubgit/HumanML3D/HumanML3D/paired_text_motion.npz"
TEST_NPZ  = "/Users/akhidre/pubgit/HumanML3D/HumanML3D/paired_text_motion_val.npz"
NORM_STATS = "motion_norm_stats.npz"   # if exists, used; otherwise computed from train npz

MOTION_LEN = 200         # fixed length for MLP outputs (frames)
NUM_JOINTS = 22
COORDS = 3
OUTPUT_DIM = MOTION_LEN * NUM_JOINTS * COORDS

MAX_TRAIN_SAMPLES =0    # 0 = use all; otherwise use first N pairs
MAX_TEST_SAMPLES = 0   # 0 = use all; otherwise use first N pairs

USE_NORMALIZATION = False   #normalize data to zero mean and unity variance

# Training hyperparams
USE_GPU = False
USE_LR_SCHEDULER = False
USE_EARLY_STOPPING = False
USE_LR_LOGGER = False  # only if you added the logger

BATCH_SIZE = 4096
EPOCHS = 75
LEARNING_RATE = 1e-4
HIDDEN_DIMS = [1024, 2048, 8192]  # list: number of neurons per hidden Dense layer

# Loss options
USE_VELOCITY_LOSS = True
LAMBDA_VEL = 1
USE_WEIGHT_DECAY = True
WEIGHT_DECAY = 1e-4

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

### Utilities Functions

In [35]:
def load_npz_pairs(npz_path, max_samples=0):
    data = np.load(npz_path, allow_pickle=True)
    z_texts = data["z_texts"]   # shape (N, 384)
    motions = data["motions"]   # dtype=object, each entry (T, J, 3)
    motion_ids = data["motion_ids"] if "motion_ids" in data.files else None

    if max_samples and max_samples > 0:
        z_texts = z_texts[:max_samples]
        motions = motions[:max_samples]
        if motion_ids is not None:
            motion_ids = motion_ids[:max_samples]

    return z_texts, motions, motion_ids

def filter_valid_motions(z_list, motions_list, ids_list=None,
                         num_joints=22, coords=3):
    """
    Filters out invalid motion sequences that do not match (T, num_joints, coords).

    Returns:
        valid_z_list, valid_motions_list, valid_ids_list (or None if no ids)
    """
    valid_z = []
    valid_motions = []
    valid_ids = [] if ids_list is not None else None

    for idx, (z, m) in enumerate(zip(z_list, motions_list)):
        arr = np.array(m)

        # Check dimensionality
        if arr.ndim != 3:
            print(f"[Filter] Skipping sample at index={idx}, shape={arr.shape} (not 3D)")
            continue

        # Check joint and coordinate dimensions
        if arr.shape[1] != num_joints or arr.shape[2] != coords:
            print(f"[Filter] Skipping sample at index={idx}, shape={arr.shape} (bad joint dims)")
            continue

        # Keep sample
        valid_z.append(z)
        valid_motions.append(arr)

        if ids_list is not None:
            valid_ids.append(ids_list[idx])

    if ids_list is not None:
        return np.array(valid_z, dtype=np.float32), valid_motions, valid_ids
    else:
        return np.array(valid_z, dtype=np.float32), valid_motions, None

def compute_mean_std_from_train_motions(motions, save_path=None):
    """
    Compute per-coordinate mean and std for a list of motion sequences.

    Args:
        motions: list or object array of motions, each shape (T, J, 3)
        save_path: str or None, optional path to save stats as .npz

    Returns:
        mean: np.array of shape (3,)
        std: np.array of shape (3,)
    """
    coords_list = []
    for m in motions:
        coords_list.append(m.reshape(-1, 3))
    all_coords = np.concatenate(coords_list, axis=0)
    mean = np.mean(all_coords, axis=0).astype(np.float32)
    std  = np.std(all_coords, axis=0).astype(np.float32) + 1e-8

    if save_path is not None:
        np.savez(save_path, mean=mean, std=std)
        print(f"Saved motion normalization stats to {save_path}")
    return mean.astype(np.float32), std.astype(np.float32)

def pad_or_truncate_motion(motion, target_len=MOTION_LEN):
    T, J, C = motion.shape
    if T == target_len:
        return motion.astype(np.float32)
    if T > target_len:
        return motion[:target_len].astype(np.float32)
    # T < target_len: pad with zeros at end
    pad_len = target_len - T
    last_frame = motion[-1][None, :, :]  # shape (1, J, C)
    pad = np.repeat(last_frame, pad_len, axis=0)  # repeat last frame
    return np.concatenate([motion.astype(np.float32), pad], axis=0)

def normalize_motion(motion, mean, std):
    # motion: (T, J, 3) -> broadcast mean/std over joints/frames
    return (motion - mean) / std

def denormalize_motion(motion_norm, mean, std):
    return motion_norm * std + mean

### Load data 

In [ ]:
print("Loading train npz:", TRAIN_NPZ)
z_train, motions_train, ids_train = load_npz_pairs(TRAIN_NPZ, max_samples=MAX_TRAIN_SAMPLES)
print("Filtering training motions...")
z_train, motions_train, ids_train = filter_valid_motions(z_train, motions_train, ids_train)
print("Final train samples:", len(z_train))

print("Loading test npz:", TEST_NPZ)
z_test, motions_test, ids_test = load_npz_pairs(TEST_NPZ, max_samples=MAX_TEST_SAMPLES)

print("Filtering test motions...")
z_test, motions_test, ids_test = filter_valid_motions(z_test, motions_test, ids_test)
print("Final test samples:", len(z_test))


print("Train captions:", z_train.shape)
print("Train motions count:", len(motions_train))
print("Train ids_count:", len(ids_train))
print("Test captions:", z_test.shape)
print("Test motions count:", len(motions_test))
print("Test ids_count:", len(ids_test))

### Compute data statistics

In [37]:
if os.path.exists(NORM_STATS):
    stats = np.load(NORM_STATS)
    mean = stats["mean"]   # shape (3,)
    std = stats["std"]
    print("Loaded normalization stats from", NORM_STATS, "mean:", mean, "std:", std)
else:
    print("Computing normalization stats from training motions...")
    # compute and save in one call
    mean, std = compute_mean_std_from_train_motions(motions_train)
print()

Computing normalization stats from training motions...



### Prepare dataset

In [ ]:
def prepare_xy(z_list, motions_objectlist, mean, std, motion_len=MOTION_LEN):
    N = len(z_list)
    X = np.array(z_list, dtype=np.float32)   # (N, 384)
    Y = np.zeros((N, motion_len, NUM_JOINTS, COORDS), dtype=np.float32)
    for i, m in enumerate(motions_objectlist):
        m_fixed = pad_or_truncate_motion(m, target_len=motion_len)
        if USE_NORMALIZATION:
            m_out = normalize_motion(m_fixed, mean, std)
        else:
            m_out = m_fixed  # leave as-is
        Y[i] = m_out
    # flatten Y for MLP regression target
    Y_flat = Y.reshape(N, -1).astype(np.float32)
    return X, Y_flat, Y  # also return 3D Y if needed

print("Preparing training tensors...")
X_train, Y_train_flat, Y_train_3d = prepare_xy(z_train, motions_train, mean, std)
print("Preparing test tensors...")
X_test, Y_test_flat, Y_test_3d = prepare_xy(z_test, motions_test, mean, std)

print("Shapes -> X_train:", X_train.shape, "Y_train_flat:", Y_train_flat.shape)
print("Shapes -> X_test :", X_test.shape, "Y_test_flat :", Y_test_flat.shape)

### MLP Model

In [39]:
tf.keras.backend.clear_session() # Clear any existing session
if USE_GPU:
    mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision policy:", mixed_precision.global_policy())
# input and model definition
inputs = tf.keras.Input(shape=(X_train.shape[1],), dtype=tf.float32, name="z_text")
x = inputs
for i, h in enumerate(HIDDEN_DIMS):
    x = tf.keras.layers.Dense(h, activation="relu", name=f"dense_{i+1}")(x)
# final linear layer -> output_dim
# IMPORTANT: cast final output to float32 for stable losses
outputs_flat = tf.keras.layers.Dense(OUTPUT_DIM, activation=None, name="output_flat", dtype="float32")(x)

# Optionally reshape inside model (but we will train with flattened y)
# outputs_reshaped = tf.keras.layers.Reshape((MOTION_LEN, NUM_JOINTS, COORDS))(outputs_flat)

model = tf.keras.Model(inputs=inputs, outputs=outputs_flat, name="MLP_Motion_Decoder")
model.summary()

# ---------------------------
# Loss: MSE + optional velocity loss + optional weight decay
# ---------------------------
def velocity_loss_from_flat(y_true_flat, y_pred_flat, motion_len=MOTION_LEN):
    # y_true_flat, y_pred_flat: (batch, motion_len * J * C)
    # reshape
    batch = tf.shape(y_true_flat)[0]
    y_true = tf.reshape(y_true_flat, (batch, motion_len, NUM_JOINTS, COORDS))
    y_pred = tf.reshape(y_pred_flat, (batch, motion_len, NUM_JOINTS, COORDS))
    # velocities: difference over time axis
    v_true = y_true[:,1:] - y_true[:,:-1]
    v_pred = y_pred[:,1:] - y_pred[:,:-1]
    return tf.reduce_mean(tf.square(v_pred - v_true))

# Custom training step via compile with custom loss wrapper
def custom_loss(y_true, y_pred):
    mse = tf.reduce_mean(tf.square(y_pred - y_true))
    if USE_VELOCITY_LOSS:
        vel = velocity_loss_from_flat(y_true, y_pred)
        loss_val = mse + LAMBDA_VEL * vel
    else:
        loss_val = mse
    return loss_val

optimizer = AdamW(learning_rate=LEARNING_RATE,weight_decay=WEIGHT_DECAY) #<-- gradient clipping
model.compile(optimizer=optimizer, loss=custom_loss, metrics=["mse"])

Model: "MLP_Motion_Decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_text (InputLayer)         [(None, 384)]             0         
                                                                 
 dense_1 (Dense)             (None, 1024)              394240    
                                                                 
 dense_2 (Dense)             (None, 2048)              2099200   
                                                                 
 dense_3 (Dense)             (None, 8192)              16785408  
                                                                 
 output_flat (Dense)         (None, 13200)             108147600 
                                                                 
Total params: 127426448 (486.09 MB)
Trainable params: 127426448 (486.09 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Training callbacks and monitors 

In [40]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,   # enables gradients & weights monitoring
    write_graph=False
)

# Reduce LR on Plateau
reduceLR_cb = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=1e-7,
    verbose=1
)

# Early stopping
earlystop_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=True,
    verbose=1
)

# Save best weights only
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=SAVE_Full_MODEL_PATH,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)
class LRTensorBoard(tf.keras.callbacks.Callback):
    def __init__(self, log_dir):
        super().__init__()
        self.file_writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        with self.file_writer.as_default():
            tf.summary.scalar("learning_rate", lr, step=epoch)

lr_logger = LRTensorBoard(log_dir=log_dir)





callbacks_list = [tensorboard_cb, checkpoint_cb]

if USE_LR_SCHEDULER:
    callbacks_list.append(reduceLR_cb)

if USE_EARLY_STOPPING:
    callbacks_list.append(earlystop_cb)

# Optional: add LR logger if you use it
if USE_LR_LOGGER:
    callbacks_list.append(lr_logger)

### Training

In [ ]:
if not from_scratch and os.path.exists(SAVE_Full_MODEL_PATH):
    print("Loading model (weights or full checkpoint)")
    model=tf.keras.models.load_model(SAVE_Full_MODEL_PATH,custom_objects={'custom_loss': custom_loss})
else:
    print("training from scratch.")
    pass  # do nothing, use existing model

tf.config.run_functions_eagerly(False)  # for performance
history = model.fit(
    X_train, Y_train_flat,
    validation_data=(X_test, Y_test_flat),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True,
    callbacks=callbacks_list,
    verbose=1
)
# plot loss curves
# ----------------------------
# 1) Combined loss plot
# ----------------------------
plt.figure(figsize=(8,5))
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Combined Loss (Pose + Velocity)')
plt.title('Training / Validation Combined Loss')
plt.legend()
plt.grid(True)
plt.show()

# ----------------------------
# 2) Pose-only MSE plot
# ----------------------------
plt.figure(figsize=(8,5))
plt.plot(history.history['mse'], label='train_mse')
plt.plot(history.history['val_mse'], label='val_mse')
plt.xlabel('Epoch')
plt.ylabel('Pose MSE')
plt.title('Training / Validation Pose MSE (Position Only)')
plt.legend()
plt.grid(True)
plt.show()

### Testing

In [ ]:
test_metrics = model.evaluate(X_test, Y_test_flat, batch_size=BATCH_SIZE, return_dict=True)
print("Test metrics:", test_metrics)

# compute velocity loss separately if needed
if USE_VELOCITY_LOSS:
    preds_flat = model.predict(X_test, batch_size=BATCH_SIZE)
    vel_loss_val = velocity_loss_from_flat(tf.constant(Y_test_flat), tf.constant(preds_flat))
    print("Test velocity loss:", float(vel_loss_val))


### Run Example

In [ ]:
sample_idx = 0 #sampling example

motion_id = ids_test[sample_idx]
print("Test motion ID:", motion_id)


z = X_test[sample_idx:sample_idx+1]
pred_flat = model.predict(z)  # shape (1, OUTPUT_DIM)
pred_3d = pred_flat.reshape(MOTION_LEN, NUM_JOINTS, COORDS)

if USE_NORMALIZATION:
    pred_real = denormalize_motion(pred_3d, mean, std)
else:
    pred_real = pred_3d

print("Predicted motion final shape:", pred_real.shape)

### Animation of Generated Motion

In [ ]:
pose=copy.deepcopy(pred_real)
#SMPL 22-joint skeleton
edges = [
    (0, 1), (1, 4), (4, 7), (7, 10),
    (0, 2), (2, 5), (5, 8), (8, 11),
    (0, 3), (3, 6), (6, 9), (9, 12), (12, 15),
    (9, 13), (13, 16), (16, 18), (18, 20),
    (9, 14), (14, 17), (17, 19), (19, 21),
]

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Create lines initially without data
lines = [ax.plot([], [], [])[0] for _ in range(pose.shape[1] - 1)]
ax.set(xlim3d=(-1, 3), xlabel='X')
ax.set(ylim3d=(-1, 1), ylabel='Y')
ax.set(zlim3d=(-1, 1), zlabel='Z')

def init(): 
    ax.cla()
    return ax,

def update_lines(frame_num, pose, lines):
    frame = pose[frame_num]
    for n in range(len(lines)):
        i, j = edges[n]
        x = [frame[i, 0], frame[j, 0]]
        y = [frame[i, 1], frame[j, 1]]
        z = [frame[i, 2], frame[j, 2]]
        lines[n].set_data_3d([z,x,y])
    return lines

ani = FuncAnimation(fig, update_lines, pose.shape[0], fargs=(pose, lines), interval=100)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
ani.save(f'/Users/akhidre/pubgit/CS230_T2Motion/animations/generate_{motion_id}_{timestamp}.mp4', writer = 'ffmpeg', fps = 30)

### Troubleshoot/Debug

In [11]:
import tensorflow as tf
print("TF GPUs:", tf.config.list_physical_devices('GPU'))

import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices())

tf.config.experimental.get_visible_devices()
tf.config.list_logical_devices('GPU')

os.path.exists(SAVE_Full_MODEL_PATH)

TF GPUs: []
2.15.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


False